In [1]:
import json
import lawquery
import pandas as pd
import os
from tqdm import tqdm

c:\Users\ngoph\.conda\envs\research\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\ngoph\.conda\envs\research\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\ngoph\.conda\envs\research\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
c:\Users\ngoph\.conda\envs\research\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Load data

In [2]:
topk = [5,10,20,50]
run_results = []

In [3]:
engines = {}
law_df = pd.read_csv('./documents/data.csv')
for i in range(len(law_df)):
    path = law_df['path'][i]
    so_hieu_van_ban = law_df['so_hieu_van_ban'][i]
    if so_hieu_van_ban not in engines:
        engines[so_hieu_van_ban] = lawquery.Engine(
            os.path.join('documents',path, 'tree.json.gz'),
        )
print("Number of engines:",len(engines))

documents = []
metadatas = []
ids = []

# take all
for engine in engines:
    results = engines[engine].query(node_type='điều')
    for result in results:
        documents.append(result.name+'\n'+result.content)
        metadatas.append({'law_id': engine, 'node_type': result.node_type, 'node_id': result.node_id})
        ids.append(result.id)

qa_df = pd.read_json('./answers_filtered.jsonl', lines=True, orient='records')

# take only use
# for index, row in qa_df.iterrows():
#     for answer in row['new_answers']:
#         q = engines[answer['law_id']].query(node_type=answer['node_type'], node_id = answer['node_id'])
#         if len(q)!=0:
#             if q[0].id in ids:
#                 continue
#             documents.append(q[0].content)
#             metadatas.append({'law_id': answer['law_id'], 'node_type': q[0].node_type, 'node_id': q[0].node_id})
#             ids.append(q[0].id)
print("Number of documents:",len(documents))

Number of engines: 15
Number of documents: 761


# TDIDF

In [4]:
from underthesea import text_normalize,word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
def format_text(text,word_segmentation=False,remove_punctuation=False):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = text_normalize(text)
    if remove_punctuation:
        text = text.translate(str.maketrans('', '', string.punctuation))
    if word_segmentation:
        text = word_tokenize(text, format="text")
    return text

In [5]:
def run(word_segmentation):
    tdidf_docs= [format_text(doc,word_segmentation) for doc in documents]
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(tdidf_docs)
    to_run = len(qa_df)
    acc ={}
    for k in topk:
        acc[k] = 0
    for i in tqdm(range(to_run)):
        row = qa_df.iloc[i]
        query = row['cauhoi']
        query = format_text(query,word_segmentation)
        query_vector = tfidf.transform([query])
        cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
        related_docs_indices = cosine_similarities.argsort()[:-topk[-1]-1:-1]
        result = { 'metadatas':[]}
        for j in related_docs_indices:
            result['metadatas'].append(metadatas[j])
        for k in topk:
            if all(r in result['metadatas'][:k] for r in row['new_answers']):
                acc[k] += 1
    # print as table
    for k in topk:
        acc[k] /= to_run
    print(acc)
    return acc

In [6]:
rr= run(False)
rr['name'] = 'tdidf'
run_results.append(rr)
rr= run(True)
rr['name'] = 'tdidf_ws'
run_results.append(rr)

100%|██████████| 4205/4205 [00:08<00:00, 513.39it/s]


{5: 0.10368608799048752, 10: 0.20095124851367419, 20: 0.3469678953626635, 50: 0.5288941736028537}


100%|██████████| 4205/4205 [00:22<00:00, 183.28it/s]

{5: 0.10939357907253269, 10: 0.1990487514863258, 20: 0.3343638525564804, 50: 0.5186682520808561}


# BM 25

In [7]:
from rank_bm25 import BM25Okapi

def run_bm25(word_segmentation,bm25_algo):
    docs= [format_text(doc,word_segmentation) for doc in documents]
    tokenized_corpus = [doc.split(" ") for doc in docs]
    bm25 = bm25_algo(tokenized_corpus)
    to_run = len(qa_df)
    acc ={}
    for k in topk:
        acc[k] = 0
    for i in tqdm(range(to_run)):
        row = qa_df.iloc[i]
        query = row['cauhoi']
        query = format_text(query,word_segmentation)
        tokenized_query = query.split(" ")
        doc_scores = bm25.get_scores(tokenized_query)
        result = { 'metadatas':[]}
        for j in doc_scores.argsort()[:-topk[-1]-1:-1]:
            result['metadatas'].append(metadatas[j])
        for k in topk:
            if all(r in result['metadatas'][:k] for r in row['new_answers']):
                acc[k] += 1
    for k in topk:
        acc[k] /= to_run
    print(acc)
    return acc

In [8]:
print("BM25Okapi")
rr=run_bm25(False,BM25Okapi)
rr['name'] = 'bm25'
run_results.append(rr)
rr=run_bm25(True,BM25Okapi)
rr['name'] = 'bm25_ws'
run_results.append(rr)

BM25Okapi


100%|██████████| 4205/4205 [00:47<00:00, 88.36it/s] 


{5: 0.07895362663495838, 10: 0.14744351961950058, 20: 0.2556480380499406, 50: 0.4485136741973841}


100%|██████████| 4205/4205 [00:56<00:00, 74.11it/s] 

{5: 0.09441141498216409, 10: 0.17455410225921522, 20: 0.29084423305588586, 50: 0.4708680142687277}


# Chroma DB

## WITH  Instructor Embedding

In [9]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from InstructorEmbedding import INSTRUCTOR

c:\Users\ngoph\.conda\envs\research\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [10]:
def run_instructor(model_name):
    model = INSTRUCTOR(model_name)
    instructor_for_q = 'Represent the legal question for retrieving evidence documents:'
    instructor_for_r = 'Represent the legal document for retrieval:'
    texts_with_instructions = []
    for doc in documents:
        texts_with_instructions.append([instructor_for_r,doc])
    customized_embeddings = model.encode(texts_with_instructions,show_progress_bar=True)
    chroma_client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
                                        persist_directory="chroma"))
    chroma_client.reset()
    collection = chroma_client.get_or_create_collection(name="law_documents")
    collection.upsert(documents=documents,metadatas=metadatas,ids=ids,embeddings=customized_embeddings.tolist())
    chroma_client.persist()
    to_run = len(qa_df)
    acc ={}
    for k in topk:
        acc[k] = 0
    for i in tqdm(range(to_run)):
        row = qa_df.iloc[i]
        question = row['cauhoi']
        query_embeddings = model.encode([[instructor_for_q,question]],show_progress_bar=False).tolist()
        result = collection.query(query_embeddings=query_embeddings, n_results=topk[-1],include=["metadatas"])
        for k in topk:
            if all(r in result['metadatas'][0][:k] for r in row['new_answers']):
                acc[k] += 1
    for k in topk:
        acc[k] /= to_run
    print(acc)
    return acc

In [11]:
rr=run_instructor('hkunlp/instructor-base')
rr['name'] = 'instructor-base'
run_results.append(rr)
rr=run_instructor('hkunlp/instructor-large')
rr['name'] = 'instructor-large'
run_results.append(rr)
rr=run_instructor('hkunlp/instructor-xl')
rr['name'] = 'instructor-xl'
run_results.append(rr)


rr=run_instructor('C:/Users/ngoph/Desktop/luanvan/model')
rr['name'] = 'instructor-base finetune'
run_results.append(rr)



load INSTRUCTOR_Transformer
max_seq_length  512


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 4205/4205 [01:27<00:00, 48.24it/s]


{5: 0.011890606420927468, 10: 0.02211652794292509, 20: 0.041617122473246136, 50: 0.09441141498216409}
load INSTRUCTOR_Transformer
max_seq_length  512


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

...

100%|██████████| 4205/4205 [12:41<00:00,  5.52it/s]


{5: 0.0187871581450654, 10: 0.031153388822829964, 20: 0.05374554102259215, 50: 0.1426872770511296}
load INSTRUCTOR_Transformer
max_seq_length  512


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 4205/4205 [01:28<00:00, 47.77it/s]

{5: 0.48323424494649225, 10: 0.5740784780023781, 20: 0.6620689655172414, 50: 0.7764565992865636}


# save result

In [12]:
result_df = pd.DataFrame(run_results)
# rename col topk
for k in topk:
    result_df.rename(columns={k: 'top'+str(k)}, inplace=True)
result_df.rename(columns={'name': 'Name',
                            'top5': 'Top_5@acc',
                            'top10': 'Top_10@acc',
                            'top20': 'Top_20@acc',
                            'top50': 'Top_50@acc',
                          }, inplace=True)
# reorder
result_df = result_df[['Name','Top_5@acc','Top_10@acc','Top_20@acc','Top_50@acc']]
result_df = result_df.round(4)
# reoder row, ws after no ws
# result_df = result_df.reindex([0,4,6,2,1,5,7,3,8,9,10])
# result_df = result_df.reindex([0,2,1,3])
result_df.to_csv('../document/data/retrieval_result.csv',index=False)
# result_df.to_csv('retrieval_result_all.csv',index=False)
result_df

,Name,Top_5@acc,Top_10@acc,Top_20@acc,Top_50@acc
0,tdidf,0.1037,0.2010,0.3470,0.5289
1,tdidf_ws,0.1094,0.1990,0.3344,0.5187
2,bm25,0.0790,0.1474,0.2556,0.4485
3,bm25_ws,0.0944,0.1746,0.2908,0.4709
4,instructor-base,0.0119,0.0221,0.0416,0.0944
5,instructor-large,0.0138,0.0247,0.0421,0.1023
6,instructor-xl,0.0188,0.0312,0.0537,0.1427
7,instructor-base finetune,0.4832,0.5741,0.6621,0.7765
